In [3]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

# model_client = OpenAIChatCompletionClient(model="gpt-4o")
from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    # model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
    # reflect_on_tool_use=True, # By default, AssistantAgent returns the tool output as the response. If your tool does not return a well-formed string in natural language format, you may want to add a reflection step within the agent by setting reflect_on_tool_use=True when creating the agent. This will allow the agent to reflect on the tool output and provide a natural language response.
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task), output_stats=True)

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
1. WebSearchAgent: Find out who the Miami Heat player with the highest points scored during the 2006-2007 season was.
2. DataAnalystAgent: Calculate the total rebounds for the top point scorer from the 2007-2008 and 2008-2009 seasons, then determine the percentage change in his rebound total between these two seasons.
[Prompt tokens: 173, Completion tokens: 89]
---------- PlanningAgent ----------

[Prompt tokens: 261, Completion tokens: 1]
---------- WebSearchAgent ----------
[FunctionCall(id='call_g90j0eo7', arguments='{"query":"Miami Heat highest points scorer 2006-2007 season"}', name='search_web_tool')]
[Prompt tokens: 328, Completion tokens: 36]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=173, completion_tokens=89), metadata={}, content='1. WebSearchAgent: Find out who the Miami Heat player with the highest points scored during the 2006-2007 season was.\n2. DataAnalystAgent: Calculate the total rebounds for the top point scorer from the 2007-2008 and 2008-2009 seasons, then determine the percentage change in his rebound total between these two seasons.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=261, completion_tokens=1), metadata={}, content='', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=32

In [9]:
def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    """You can see from the conversation log that the Planning Agent always speaks immediately after the specialized agents."""
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    Wait for the results from other team members, confirm the results can answer the question from user.
    If the results is correct, then summarize the findings and end with "TERMINATE". Otherwise give some suggestions to your team members to start the tasks.
    """,
)

# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task), output_stats=True)

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.
2. DataAnalystAgent: Calculate the percentage change in total rebounds for the top points scorer between the 2007-2008 and 2008-2009 seasons.
[Prompt tokens: 206, Completion tokens: 74]
---------- PlanningAgent ----------

[Prompt tokens: 279, Completion tokens: 1]
---------- PlanningAgent ----------
Let's wait for their results to proceed with further steps.
[Prompt tokens: 279, Completion tokens: 13]
---------- WebSearchAgent ----------
[FunctionCall(id='call_qys9t637', arguments='{"query":"Miami Heat player highest points in 2006-2007 season"}', name='search_web_tool')]
[Prompt tokens: 325, Completion tokens: 37]
---------- WebSearchAgent ----------


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=206, completion_tokens=74), metadata={}, content='1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. DataAnalystAgent: Calculate the percentage change in total rebounds for the top points scorer between the 2007-2008 and 2008-2009 seasons.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=279, completion_tokens=1), metadata={}, content='', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=279, completion_tokens=13), metadata={}, content="Let's wait for their results to proceed with 

In [10]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season.
2. DataAnalystAgent : Calculate the percentage change in total rebounds for the identified player between the 2007-2008 and 2008-2009 seasons.
---------- UserProxyAgent ----------
approve
---------- WebSearchAgent ----------
[FunctionCall(id='call_9896hgfq', arguments='{"query":"Miami Heat highest points scorer 2006-2007 season"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_web_tool',

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=206, completion_tokens=73), metadata={}, content='1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. DataAnalystAgent : Calculate the percentage change in total rebounds for the identified player between the 2007-2008 and 2008-2009 seasons.', type='TextMessage'), UserInputRequestedEvent(source='UserProxyAgent', models_usage=None, metadata={}, request_id='af81bc81-767f-4157-b9a6-87cb2688ae78', content='', type='UserInputRequestedEvent'), TextMessage(source='UserProxyAgent', models_usage=None, metadata={}, content='approve', type='TextMessage'), ToolCallRequestEvent(source='Web